# 네이버 자주 본 뉴스 크롤링

1) 수집내용

    1) 많이본뉴스–섹션별(정치~IT/과학)Top5기사제목,신문사,뷰 
    2) 해당 기사별 기사 내용, 리액션 (좋아요 ~ 후속기사 원해요)
    
2) 수집방법(택1)

    1) [기본] Requests , BeautifulSoup, Selenium
    2) [심화] Requests, BeautifulSoup (+ 멀티프로세싱)
    
3) 수집범위 및 저장

    1) 2019년7월21일~2020년8월20일(동작가능,실제구동x)
    2) 하나의 파일로 저장 (방식 자유)
    3) Ex)총6섹션*Top5*365일=10950rows

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from IPython.display import Image
import os
from pathlib import Path
import glob

import datetime

## 날짜 String 생성

In [2]:
start_date = datetime.datetime(2019, 7, 21, 3, 22, 32)
# start_date.strftime("%Y%m%d %Y-%m-%d 시간 표시외 %H:%M:%S 아무거나 넣어도 됩니다.") #strftime 활용 예시

In [3]:
def get_date_string(start_date, period=365):
    # 20190721 형태의 Date String List 생성
    return [
        (start_date + datetime.timedelta(days=day)).strftime("%Y%m%d")
        for day in range(period)
    ]

In [4]:
date_string_list = get_date_string(start_date, 366)

In [5]:
date_string_list[0], date_string_list[-1] # 생성 완료

('20190721', '20200720')

## Part 1. Request를 이용한 크롤링

In [6]:
def get_top_news(date):
    """
    해당 날짜의 자주 본 뉴스 25개에 대한 정보를 반환하는 함수입니다.
    """
    url = f"https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&date={date}" # 이부분을 잘 채워넣어주세요 (네이버 뉴스 자주보는 뉴스 링크에서 date를 제외한 부분)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    result = []
    
    count = 0
    title_link = soup.select("li > dl > dt > a")
    publisher_main = soup.select("li > dl > dd > span:nth-child(2n)")
    publisher = soup.select("li > dl > dt > span:nth-child(2n)")
    view_main = soup.select("li > dl > dd> i.count_view")
    view = soup.select("li > dl > dt> i.count_view")
    while title_link:
        temp = title_link.pop(0)
        title = temp.text
        link = temp["href"]
        if count%5 == 0:
            temp = publisher_main.pop(0)
            publish = temp.text.strip()
            temp = view_main.pop(0)
            views = temp.text
        else:
            temp = publisher.pop(0)
            publish = temp.text.strip()
            temp = view.pop(0)
            views = temp.text
        temp_list = [title,link,publish,views]
        result.append(temp_list)
        count +=1
    # soup.select를 잘 활용하여
    # 1. 신문 제목
    # 2. 기사 링크 (a tag의 href 속성) 
    # 3. 신문사명
    # 4. View
    
    return result

In [59]:
# 1년치 전체를 하는 것이 아닌 앞 20일치만 테스트 해보기 위함 (개인적으로 바꿔도 무방)
news_data = []
for date in date_string_list:
     news_data.extend(get_top_news(date)) # 결과로 나온 25개를 이어 붙임

In [60]:
len(news_data)

600

In [61]:
news_data[:5] # 결과 예시

[['[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=032&aid=0002952683&date=20190721&type=1&rankingSectionId=100&rankingSeq=1',
  '경향신문',
  '201,039'],
 ['조국 폭풍페북, 日주장 정면반박…"친일파" 표현은 野 반발',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=008&aid=0004251344&date=20190721&type=1&rankingSectionId=100&rankingSeq=2',
  '머니투데이',
  '185,396'],
 ['조국, 연일 對日 \'항전\' 주문…"겁먹고 쫄지말자…싸워 이겨…',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=001&aid=0010969325&date=20190721&type=1&rankingSectionId=100&rankingSeq=3',
  '연합뉴스',
  '130,198'],
 ['[김순덕의 도발]복수를 하려면 아일랜드처럼!',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=020&aid=0003230442&date=20190721&type=1&rankingSectionId=100&rankingSeq=4',
  '동아일보',
  '120,897'],
 ['조국, 또 페북에 反日 선전전..."文정부, 서희·이순신 역할…',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=023&aid=0003462170&date=20190721&type=1&rankingSectio

In [62]:
# 결과물을 데이터 프레임으로 변환 및 Column Name 부여
df_top_news = pd.DataFrame(news_data, columns=["title", "url", "press", "views"])

In [63]:
df_top_news

,title,url,press,views
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,경향신문,"201,039"
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,머니투데이,"185,396"
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,연합뉴스,"130,198"
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,동아일보,"120,897"
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,조선일보,"119,463"
5,[취재파일] 日 경제 보복…이재용의 침묵과 최태원의 당당함,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,SBS,"157,764"
6,"불매운동에 놀란 유니클로 대표 ""추가 사과할 것""",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,머니투데이,"136,808"
7,"일본 여행 갔다고 '공개 망신'…""성숙한 불매운동 필요""",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,SBS,"126,836"
8,"""예·적금 가입 서두르세요""···2%대 이자 사라진다",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,서울경제,"125,516"
9,'주식투자로 50평 아파트' 70대 노부인의 행동재무학 재테크,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,머니투데이,"122,556"


In [64]:
# URL 앞에  "https://news.naver.com" 붙이기
df_top_news.url = "https://news.naver.com"+df_top_news.url # 여기를 채워넣어주세요

In [67]:
df_top_news

,title,url,press,views
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,"201,039"
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,"185,396"
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,"130,198"
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,"120,897"
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"119,463"
5,[취재파일] 日 경제 보복…이재용의 침묵과 최태원의 당당함,https://news.naver.com/main/ranking/read.nhn?m...,SBS,"157,764"
6,"불매운동에 놀란 유니클로 대표 ""추가 사과할 것""",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,"136,808"
7,"일본 여행 갔다고 '공개 망신'…""성숙한 불매운동 필요""",https://news.naver.com/main/ranking/read.nhn?m...,SBS,"126,836"
8,"""예·적금 가입 서두르세요""···2%대 이자 사라진다",https://news.naver.com/main/ranking/read.nhn?m...,서울경제,"125,516"
9,'주식투자로 50평 아파트' 70대 노부인의 행동재무학 재테크,https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,"122,556"


## Part 2. Selenium을 이용한 크롤링

In [70]:
driver_path = os.path.join(Path().absolute(), "chromedriver")# driver 경로 설정

In [163]:
driver = webdriver.Chrome(driver_path) # Chrome driver 실행

In [128]:
df_top_news.url[:10]

0    https://news.naver.com/main/ranking/read.nhn?m...
1    https://news.naver.com/main/ranking/read.nhn?m...
2    https://news.naver.com/main/ranking/read.nhn?m...
3    https://news.naver.com/main/ranking/read.nhn?m...
4    https://news.naver.com/main/ranking/read.nhn?m...
5    https://news.naver.com/main/ranking/read.nhn?m...
6    https://news.naver.com/main/ranking/read.nhn?m...
7    https://news.naver.com/main/ranking/read.nhn?m...
8    https://news.naver.com/main/ranking/read.nhn?m...
9    https://news.naver.com/main/ranking/read.nhn?m...
Name: url, dtype: object

In [164]:
# NEWS_TEST_RANGE = 10
for idx, news_url in enumerate(df_top_news.url):
    # 드라이버 내에서 해당 URL로 이동
    driver.get(news_url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    ## BeautifulSoup 혹은 driver.find_element[s]_by_css_selector 을 이용하여 정보 파싱
    # +기사 내용
    content = soup.select("#articleBodyContents")[0].get_text().replace('\n', " ") 

    # +5가지 리액션 (좋아요, 훈훈해요, 슬퍼요, 화나요, 후속기사원해요) 투표 수
    key_list = ['좋아요', '훈훈해요', '슬퍼요', '화나요', '후속기사원해요']
    value_list = []
    for i in range(5):
        temp = driver.find_elements_by_css_selector("#spiLayer > div._reactionModule.u_likeit > ul > li")[i].text
        value_list.append(''.join(list(filter(str.isdigit,temp))))
    reaction_dict = {k:v for k,v in zip(key_list, value_list)}
    
    
    
    # 예시로 content라는 변수에 기사 내용을 담고 Column "content"에 해당 내용 저장
    df_top_news.loc[idx,"content"] = content
    for k,v in reaction_dict.items():
        df_top_news.loc[idx,k] = v
driver.close()

In [165]:
df_top_news.head(5)

,title,url,press,views,content,좋아요,훈훈해요,슬퍼요,화나요,후속기사원해요
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,"201,039",// flash 오류를 우회하기 위한 함수 추가 function _flas...,131,16,17,2605,121
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,"185,396",// flash 오류를 우회하기 위한 함수 추가 function _flas...,1710,32,13,6058,48
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,"130,198",// flash 오류를 우회하기 위한 함수 추가 function _flas...,1799,19,13,9120,30
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,"120,897",// flash 오류를 우회하기 위한 함수 추가 function _flas...,3017,19,16,623,37
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"119,463",// flash 오류를 우회하기 위한 함수 추가 function _flas...,374,13,14,11468,38


In [177]:
# Content 앞 데이터 전처리
def content_replace(x):
    if type(x) != str:
        return
    x = x.replace('// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}', " ")
    x = x.replace('\t'," ")
    return x
df_top_news.content = df_top_news.content.map(content_replace)

In [182]:
df_top_news.head(5)

,title,url,press,views,content,좋아요,훈훈해요,슬퍼요,화나요,후속기사원해요
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,"201,039",중 3 때 오빠와 장관상이어 고3 때도 ‘장함모’ 활동으로 자원봉사대...,131,16,17,2605,121
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,"185,396","[머니투데이 김성휘 ,백지수 기자] [[the300]징용판결 ...",1710,32,13,6058,48
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,"130,198","""文정부, 서희와 이순신 역할 함께 수행…지레 겁먹지 말아야""""...",1799,19,13,9120,30
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,"120,897",친일잔재를 청산하고 한번도 경험하지 못한 나라로 가는 것이 목적...,3017,19,16,623,37
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"119,463","""문재인 정부, 국익 수호 위해 '서희' '이순신' 역할 함께 수행""...",374,13,14,11468,38


> 본 데이터 프레임은 예시로, 리액션 데이터를 담지 않았습니다. **과제에는 리액션 데이터가 반드시 담겨져있어야합니다.**

# 데이터 저장

> 파일 형태로 크롤링한 데이터를 저장

In [ ]:
df_top_news.to_csv("top_news.csv")